In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms

In [7]:
data_dir = r'C:\Users\Utilisateur\Documents\GitHub\Post-Hurricane-Imagery-on-CNN'

trans = transforms.Compose([transforms.ToTensor(),transforms.Resize((150,150)), transforms.Pad(1)])


#Train
train = datasets.ImageFolder(data_dir + "/train_another",transform = trans)

#Validation
validation = datasets.ImageFolder(data_dir + "/validation_another",transform = trans)

#train loader
train_loader = torch.utils.data.DataLoader(train, 20, shuffle = True, num_workers = 4, pin_memory = True)

#test loader
val_loader = torch.utils.data.DataLoader(validation, 20, shuffle = False, num_workers = 4, pin_memory = True)

In [17]:
#Fonction permettant d'obtenir l'accuracy des modèles
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [18]:
#Création d'une classe de suivis des modèles avec prediction sur les données de validation
#et metrics d'analyse
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [27]:
#Model de l'article en pytorch avec comme base la classe de suivi des modèles
class NewCNN(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3,64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(64,128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Conv2d(128,128, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),

            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(128*15*15, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
            nn.Sigmoid()
        )
    
    def forward(self, xb):
        return self.network(xb)

model = NewCNN()

In [23]:
#Evaluation du modèle pour l'entrainement
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = []
    step = 0
    for batch in val_loader:
        outputs.append(model.validation_step(batch))
        step += 1
        if step == 50:
            break
        
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        step = 0
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            step += 1
            if step == 100:
                break 
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history, model

In [22]:
model

NewCNN(
  (network): Sequential(
    (0): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Dropout(p=0.5, inplace=False)
    (11): Linear(in_features=41472, out_features=512, bias=True)
    (12): ReLU()
    (13): Linear(in_features=512, out_features=2, bias=True)
    (14): Sigmoid()
  )
)

In [28]:
num_epochs = 20
opt_func = torch.optim.RMSprop
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 0.7391, val_loss: 0.7831, val_acc: 0.0010
Epoch [1], train_loss: 0.6069, val_loss: 0.6132, val_acc: 0.6730
Epoch [2], train_loss: 0.5121, val_loss: 0.5446, val_acc: 0.7580
Epoch [3], train_loss: 0.4879, val_loss: 0.7452, val_acc: 0.5410
Epoch [4], train_loss: 0.4650, val_loss: 0.3768, val_acc: 0.9390
Epoch [5], train_loss: 0.4481, val_loss: 0.3676, val_acc: 0.9480
Epoch [6], train_loss: 0.4323, val_loss: 0.4112, val_acc: 0.8990
Epoch [7], train_loss: 0.4231, val_loss: 0.3499, val_acc: 0.9670
Epoch [8], train_loss: 0.4133, val_loss: 0.4134, val_acc: 0.8980
Epoch [9], train_loss: 0.4040, val_loss: 0.3620, val_acc: 0.9500
Epoch [10], train_loss: 0.4011, val_loss: 0.4090, val_acc: 0.9070
Epoch [11], train_loss: 0.3977, val_loss: 0.5155, val_acc: 0.7900
Epoch [12], train_loss: 0.3991, val_loss: 0.3425, val_acc: 0.9720
Epoch [13], train_loss: 0.3932, val_loss: 0.3488, val_acc: 0.9660
Epoch [14], train_loss: 0.3870, val_loss: 0.3807, val_acc: 0.9320
Epoch [15], train_lo

In [30]:
num_epochs = 30
opt_func = torch.optim.Adam
lr = 1e-4
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 0.3629, val_loss: 0.3364, val_acc: 0.9800
